# 🗑️ Cleanup ML Deployment

This notebook removes all ML pipeline resources from the OpenShift cluster:
- InferenceService (`alert-recommender`)
- ServingRuntime (`mlserver-sklearn`)
- MinIO deployment and services
- RBAC (Role and RoleBinding)
- Secrets (`minio-secret`, `storage-config`)

**⚠️ WARNING**: This will delete all ML infrastructure and stored models!

## Setup

In [ ]:
import os
from kubernetes import client, config
from kubernetes.client.rest import ApiException

In [ ]:
# Configuration
NAMESPACE = os.getenv('NAMESPACE', 'spending-transaction-monitor')

print(f'🎯 Target namespace: {NAMESPACE}')
print(f'⚠️  This will delete all ML resources in this namespace!')

In [ ]:
# Load Kubernetes configuration
try:
    config.load_incluster_config()
    print('✅ Using in-cluster configuration')
except:
    config.load_kube_config()
    print('✅ Using local kubeconfig')

api_client = client.ApiClient()

## 1. Delete InferenceService

In [ ]:
print('\n🗑️  Deleting InferenceService...')

custom_api = client.CustomObjectsApi(api_client)

try:
    custom_api.delete_namespaced_custom_object(
        group='serving.kserve.io',
        version='v1beta1',
        namespace=NAMESPACE,
        plural='inferenceservices',
        name='alert-recommender'
    )
    print('✅ InferenceService "alert-recommender" deleted')
except ApiException as e:
    if e.status == 404:
        print('ℹ️  InferenceService "alert-recommender" not found (already deleted)')
    else:
        print(f'❌ Error deleting InferenceService: {e}')

## 2. Delete ServingRuntime

In [ ]:
print('\n🗑️  Deleting ServingRuntime...')

try:
    custom_api.delete_namespaced_custom_object(
        group='serving.kserve.io',
        version='v1alpha1',
        namespace=NAMESPACE,
        plural='servingruntimes',
        name='mlserver-sklearn'
    )
    print('✅ ServingRuntime "mlserver-sklearn" deleted')
except ApiException as e:
    if e.status == 404:
        print('ℹ️  ServingRuntime "mlserver-sklearn" not found (already deleted)')
    else:
        print(f'❌ Error deleting ServingRuntime: {e}')

## 3. Delete MinIO Resources

In [ ]:
print('\n🗑️  Deleting MinIO deployment...')

apps_v1 = client.AppsV1Api(api_client)

try:
    apps_v1.delete_namespaced_deployment(
        name='minio',
        namespace=NAMESPACE
    )
    print('✅ MinIO deployment deleted')
except ApiException as e:
    if e.status == 404:
        print('ℹ️  MinIO deployment not found (already deleted)')
    else:
        print(f'❌ Error deleting MinIO deployment: {e}')

In [ ]:
print('\n🗑️  Deleting MinIO services...')

core_v1 = client.CoreV1Api(api_client)

for service_name in ['minio-service', 'minio']:
    try:
        core_v1.delete_namespaced_service(
            name=service_name,
            namespace=NAMESPACE
        )
        print(f'✅ Service "{service_name}" deleted')
    except ApiException as e:
        if e.status == 404:
            print(f'ℹ️  Service "{service_name}" not found (already deleted)')
        else:
            print(f'❌ Error deleting service "{service_name}": {e}')

In [ ]:
print('\n🗑️  Deleting MinIO PVC...')

try:
    core_v1.delete_namespaced_persistent_volume_claim(
        name='minio-pvc',
        namespace=NAMESPACE
    )
    print('✅ MinIO PVC "minio-pvc" deleted')
except ApiException as e:
    if e.status == 404:
        print('ℹ️  MinIO PVC "minio-pvc" not found (already deleted)')
    else:
        print(f'❌ Error deleting MinIO PVC: {e}')

## 4. Delete Secrets

In [ ]:
print('\n🗑️  Deleting secrets...')

for secret_name in ['minio-secret', 'storage-config']:
    try:
        core_v1.delete_namespaced_secret(
            name=secret_name,
            namespace=NAMESPACE
        )
        print(f'✅ Secret "{secret_name}" deleted')
    except ApiException as e:
        if e.status == 404:
            print(f'ℹ️  Secret "{secret_name}" not found (already deleted)')
        else:
            print(f'❌ Error deleting secret "{secret_name}": {e}')

## 5. Delete RBAC Resources (Requires Admin Privileges)

**⚠️ Note**: RBAC deletion requires cluster admin or namespace admin permissions.
The service account used by this notebook may not have these permissions.

If you see 403 Forbidden errors below, delete RBAC manually using:
```bash
oc delete rolebinding ml-deployment-manager-binding -n spending-transaction-monitor
oc delete role ml-deployment-manager -n spending-transaction-monitor
```

In [ ]:
print('\n🗑️  Deleting RoleBinding...')

rbac_v1 = client.RbacAuthorizationV1Api(api_client)

try:
    rbac_v1.delete_namespaced_role_binding(
        name='ml-deployment-manager-binding',
        namespace=NAMESPACE
    )
    print('✅ RoleBinding "ml-deployment-manager-binding" deleted')
except ApiException as e:
    if e.status == 404:
        print('ℹ️  RoleBinding "ml-deployment-manager-binding" not found (already deleted)')
    elif e.status == 403:
        print('⚠️  Permission denied: Cannot delete RoleBinding (requires admin privileges)')
        print(f'   Run manually: oc delete rolebinding ml-deployment-manager-binding -n {NAMESPACE}')
    else:
        print(f'❌ Error deleting RoleBinding: {e}')

In [ ]:
print('\n🗑️  Deleting Role...')

try:
    rbac_v1.delete_namespaced_role(
        name='ml-deployment-manager',
        namespace=NAMESPACE
    )
    print('✅ Role "ml-deployment-manager" deleted')
except ApiException as e:
    if e.status == 404:
        print('ℹ️  Role "ml-deployment-manager" not found (already deleted)')
    else:
        print(f'❌ Error deleting Role: {e}')

## 6. Delete ServiceAccount (Optional)

In [ ]:
print('\n🗑️  Deleting ServiceAccount...')

# Only delete if it's not used by other resources
DELETE_SA = False  # Set to True to delete the ServiceAccount

if DELETE_SA:
    try:
        core_v1.delete_namespaced_service_account(
            name='alert-recommendation',
            namespace=NAMESPACE
        )
        print('✅ ServiceAccount "alert-recommendation" deleted')
    except ApiException as e:
        if e.status == 404:
            print('ℹ️  ServiceAccount "alert-recommendation" not found (already deleted)')
        else:
            print(f'❌ Error deleting ServiceAccount: {e}')
else:
    print('ℹ️  Skipping ServiceAccount deletion (DELETE_SA=False)')

## 7. Verify Cleanup

In [ ]:
print('\n🔍 Verifying cleanup...')
print(f'\nNamespace: {NAMESPACE}')
print('=' * 60)

# Check InferenceServices
try:
    isvcs = custom_api.list_namespaced_custom_object(
        group='serving.kserve.io',
        version='v1beta1',
        namespace=NAMESPACE,
        plural='inferenceservices'
    )
    isvc_count = len(isvcs.get('items', []))
    print(f'InferenceServices: {isvc_count}')
    if isvc_count > 0:
        for item in isvcs['items']:
            print(f'  - {item["metadata"]["name"]}')
except Exception as e:
    print(f'InferenceServices: Error - {e}')

# Check ServingRuntimes
try:
    srs = custom_api.list_namespaced_custom_object(
        group='serving.kserve.io',
        version='v1alpha1',
        namespace=NAMESPACE,
        plural='servingruntimes'
    )
    sr_count = len(srs.get('items', []))
    print(f'ServingRuntimes: {sr_count}')
    if sr_count > 0:
        for item in srs['items']:
            print(f'  - {item["metadata"]["name"]}')
except Exception as e:
    print(f'ServingRuntimes: Error - {e}')

# Check Deployments
try:
    deps = apps_v1.list_namespaced_deployment(namespace=NAMESPACE)
    minio_deps = [d.metadata.name for d in deps.items if 'minio' in d.metadata.name.lower()]
    print(f'MinIO Deployments: {len(minio_deps)}')
    for dep in minio_deps:
        print(f'  - {dep}')
except Exception as e:
    print(f'Deployments: Error - {e}')

# Check Secrets
try:
    secrets = core_v1.list_namespaced_secret(namespace=NAMESPACE)
    ml_secrets = [s.metadata.name for s in secrets.items if s.metadata.name in ['minio-secret', 'storage-config']]
    print(f'ML Secrets: {len(ml_secrets)}')
    for secret in ml_secrets:
        print(f'  - {secret}')
except Exception as e:
    print(f'Secrets: Error - {e}')

# Check RBAC
try:
    roles = rbac_v1.list_namespaced_role(namespace=NAMESPACE)
    ml_roles = [r.metadata.name for r in roles.items if 'ml-deployment' in r.metadata.name]
    print(f'ML Roles: {len(ml_roles)}')
    for role in ml_roles:
        print(f'  - {role}')
except Exception as e:
    print(f'Roles: Error - {e}')

print('\n' + '=' * 60)
print('✅ Cleanup verification complete!')

## Summary

### What was deleted:
1. ✅ InferenceService: `alert-recommender`
2. ✅ ServingRuntime: `mlserver-sklearn`
3. ✅ MinIO Deployment and Services
4. ✅ MinIO PVC (persistent storage)
5. ✅ Secrets: `minio-secret`, `storage-config`
6. ✅ RBAC: Role and RoleBinding

### What was NOT deleted:
- ServiceAccount: `alert-recommendation` (optional, set `DELETE_SA=True` to remove)
- Namespace: `spending-transaction-monitor`

### To redeploy:
Run notebooks 1, 2, and 3 again to recreate the ML pipeline.